In [1]:
from pathlib import Path

import mmtbx.f_model
import mmtbx.model
import cctbx.crystal
import cctbx.xray
import iotbx

import sys
sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
import miller_ops
from u_aniso import read_non_altloc_structure
sys.path.append(str(Path(Path.home(), "Documents/xray/data/cifs/scripts")))
import generate_fmodel

In [19]:
pdb_file = Path(Path.home(), "Documents/xray/data/pdbs/3k0m/3k0m.pdb")
n_state = 1

In [20]:
# crystal_symmetry = cctbx.xray.crystal.symmetry(
#     unit_cell=(114.968, 54.622, 45.194, 90.000, 101.675, 90.000),
#     space_group_symbol="C 1 2 1"
# )

# xray_structure = iotbx.pdb.input(str(pdb_file)).xray_structure_simple(
#     crystal_symmetry=crystal_symmetry
# )
xray_structure = read_non_altloc_structure(pdb_file)

n_scatt = xray_structure.scatterers().size()
n_scatt_per_state = int(n_scatt/n_state)

for i in range(n_state):
    for j in range(n_scatt_per_state):
        xray_structure.scatterers()[i*n_scatt_per_state+j].occupancy = 1/n_state

        # xray_structure.scatterers()[i*n_scatt_per_state+j].b = 15

# f_model = xray_structure.structure_factors(
#     d_min=1.5
# ).f_calc().amplitudes()
# print(f_model.d_min())

# flags = f_model.generate_r_free_flags_basic()
# status = generate_fmodel.get_status_array(flags)


In [21]:
f_obs_file = Path(Path.home(), "Documents/xray/data/cifs/3k0m/3k0m.cif")
f_obs = miller_ops.get_miller_array(
    f_obs_file=f_obs_file,
    label="_refln.intensity_meas"
)
f_obs = miller_ops.clean_miller_array(f_obs)

# Set flags from file.
status_array = miller_ops.get_miller_array(
    f_obs_file=f_obs_file,
    label="_refln.status"
)
flags = status_array.customized_copy(data=status_array.data()=="f")
f_obs, flags = f_obs.common_sets(other=flags)

In [22]:
xray_structure.scatterers().flags_set_grads(
    state=False
)
xray_structure.scatterers().flags_set_grad_site(
    iselection=xray_structure.all_selection().iselection()
)
xray_structure.scatterers().flags_set_grad_occupancy(
    iselection=xray_structure.all_selection().iselection()
)

f_model_manager = mmtbx.f_model.manager(
    xray_structure=xray_structure,
    f_obs=f_obs,
    r_free_flags=flags,
    target_name="ml"
)

fmodels = mmtbx.fmodels(fmodel_xray=f_model_manager)
fmodels.update_xray_structure(
    xray_structure=xray_structure,
    update_f_calc=True
)

f_model_manager.update_all_scales(
    apply_scale_k1_to_f_obs=True,
    remove_outliers=True
)

work = f_model_manager.r_work()
free = f_model_manager.r_free()
# grads = fmodels_target_and_gradients.gradients()

In [9]:
work, free

(0.15646937756468463, 0.17065692862185558)

In [15]:
generate_fmodel.write_cif(f_model, status, Path(Path.home(), "Documents/xray/tmp/test.cif"))

In [23]:
print(type(xray_structure))
print(xray_structure.show_summary())
print(xray_structure.show_scatterers())

<class 'cctbx.xray.structure.structure'>
Number of scatterers: 2733
At special positions: 0
Unit cell: (42.419, 51.741, 88.639, 90, 90, 90)
Space group: P 21 21 21 (No. 19)
Unit cell: (42.419, 51.741, 88.639, 90, 90, 90)
Space group: P 21 21 21 (No. 19)
Label, Scattering, Multiplicity, Coordinates, Occupancy, Uiso, Ustar as Uiso
pdb=" N   VAL A   2 " N      4 (-0.4509 -0.0591 -0.2102) 1.00 [ - ] 0.5650
     u_cart =  0.503  0.545  0.647 -0.178 -0.077  0.002
pdb=" CA  VAL A   2 " C      4 (-0.4691 -0.0352 -0.2103) 1.00 [ - ] 0.5152
     u_cart =  0.437  0.518  0.590 -0.139 -0.046 -0.023
pdb=" C   VAL A   2 " C      4 (-0.4494 -0.0124 -0.2049) 1.00 [ - ] 0.4230
     u_cart =  0.325  0.513  0.432 -0.226 -0.076 -0.126
pdb=" O   VAL A   2 " O      4 (-0.4547  0.0095 -0.2097) 1.00 [ - ] 0.4760
     u_cart =  0.253  0.662  0.513 -0.170 -0.058 -0.051
pdb=" CB  VAL A   2 " C      4 (-0.4990 -0.0379 -0.2008) 1.00 [ - ] 0.5708
     u_cart =  0.508  0.546  0.659 -0.221 -0.077 -0.076
pdb=" HA  VAL 

In [34]:
scatterers = xray_structure.scatterers()

In [39]:
xray_structure_copy = xray_structure.deep_copy_scatterers()

In [43]:
scatterers_copy = xray_structure_copy.scatterers()

In [48]:
scatterers_copy[0].occupancy = 0.5


In [49]:
scatterers[0].occupancy, scatterers_copy[0].occupancy

(1.0, 0.5)